In [1]:
import warnings
from cryptography.utils import CryptographyDeprecationWarning
warnings.filterwarnings("ignore", category=CryptographyDeprecationWarning)  # Suppress specific deprecation warnings

import os  # Import os for file and directory operations
import cv2  # Import OpenCV for image processing
import pandas as pd  # Import pandas for data handling
from skimage.feature import hog  # Import HOG feature extractor from skimage
from skimage import exposure  # Import exposure module from skimage for image adjustments
import matplotlib.pyplot as plt  # Import matplotlib for saving images
import numpy as np

def extract_hog_features(directory, output_directory):
    image_files = [f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))]
    hog_features = []

    for file in image_files:
        image_path = os.path.join(directory, file)
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        
        if image is None:
            print(f"Warning: {file} could not be read.")
            continue

        fd, hog_image = hog(image, orientations=9, pixels_per_cell=(8, 8),
                            cells_per_block=(2, 2), block_norm='L2-Hys', visualize=True)
        hog_image_rescaled = exposure.rescale_intensity(hog_image, in_range=(0, 10))
        hog_image_path = os.path.join(output_directory, f"HOG_{file}")
        plt.imsave(hog_image_path, hog_image_rescaled, cmap='gray')
        hog_features.append([file] + fd.tolist())

    hog_df = pd.DataFrame(hog_features)
    hog_df.to_csv(os.path.join(output_directory, "FeaturesHOG.csv"), index=False)

def extract_sift_features(directory, output_directory):
    image_files = [f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))]
    sift = cv2.SIFT_create()
    sift_features = []

    for file in image_files:
        image_path = os.path.join(directory, file)
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        
        if image is None:
            print(f"Warning: {file} could not be read.")
            continue

        keypoints, descriptors = sift.detectAndCompute(image, None)
        if descriptors is not None:
            descriptors = descriptors.flatten()
            sift_features.append([file] + descriptors.tolist())

    sift_df = pd.DataFrame(sift_features)
    sift_df.to_csv(os.path.join(output_directory, "FeaturesSIFT.csv"), index=False)

def extract_harris_features(directory, output_directory):
    image_files = [f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))]
    harris_features = []

    for file in image_files:
        image_path = os.path.join(directory, file)
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        
        if image is None:
            print(f"Warning: {file} could not be read.")
            continue

        harris_response = cv2.cornerHarris(image, blockSize=2, ksize=3, k=0.04)
        harris_response = cv2.normalize(harris_response, None, 0, 255, cv2.NORM_MINMAX).flatten()
        harris_features.append([file] + harris_response.tolist())

    harris_df = pd.DataFrame(harris_features)
    harris_df.to_csv(os.path.join(output_directory, "FeaturesHARRIS.csv"), index=False)

if __name__ == "__main__":
    input_directory = "SegmentedImages"
    output_directory = "FeatureOutputs"
    os.makedirs(output_directory, exist_ok=True)

    extract_hog_features(input_directory, output_directory)
    extract_sift_features(input_directory, output_directory)
    extract_harris_features(input_directory, output_directory)
